<a href="https://colab.research.google.com/github/junhyeokk/boomhill24/blob/main/data/kart-data-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [264]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [265]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [266]:
def mse(img): # Error between input image and white image
  return np.sum((255-img)**2)/(np.shape(img)[0]*np.shape(img)[1])

In [267]:
def is_pushed(img): # image is pushed image if error is lager than threshold(200) (check image is almost white)
                    # when not pushed, image is almost white
  return 1 if mse(img) > 200 else 0

In [268]:
cap = cv2.VideoCapture("/content/gdrive/MyDrive/m2.mp4")

영상 추출 관련 기본 데이터

In [269]:
fps = round(cap.get(cv2.CAP_PROP_FPS))
t = 30          # Start time
end_sec = 110   # End time
interval = 1    # Time interval
# 단위는 초

print(f"fps : {fps}")

fps : 60


In [270]:
# top left position of key
# up, left, right, ctrl, shift, down 
key_pos = [(1265,914), (1188,992), (1343,992), (1092,992), (1092,914), (1265,992)]

key_size = 20

In [271]:
while t <= end_sec:
  cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
  ret,frame = cap.read()

  if not ret:
    print("영상 시간 초과 또는 영상 관련 오류")

  # 키 입력 추출
  key_array = [is_pushed(frame[index[1]:index[1]+key_size,index[0]:index[0]+key_size]) for index in key_pos]

  keystring = ''.join(str(key) for key in key_array)
  print(f"key string at {t} is {keystring}")

  cropped_img = frame[:1080,:1440]
  keylog_img = frame[881:1080,1056:1440]
  cv2.imwrite(f"/content/gdrive/MyDrive/kart_cap/kart_image{t}.jpg", cropped_img)

  # 키로그와 key string 비교 위해 keylog 이미지 저장
  #cv2.imwrite(f"/content/gdrive/MyDrive/kart_cap/keylog_image{t}.jpg", keylog_img)

  # csv 형식으로 저장 (해당 화면 이미지 파일, 다음 키 입력)
  # 이미지 파일에서 키 뷰어부분 마스킹
  t += interval

key string at 30 is 000000
key string at 31 is 000000
key string at 32 is 000000
key string at 33 is 000000
key string at 34 is 000000
key string at 35 is 000000
key string at 36 is 000000
key string at 37 is 000000
key string at 38 is 000000
key string at 39 is 000000
key string at 40 is 000000
key string at 41 is 000000
key string at 42 is 000000
key string at 43 is 000000
key string at 44 is 000000
key string at 45 is 000000
key string at 46 is 000000
key string at 47 is 000000
key string at 48 is 000000
key string at 49 is 110000
key string at 50 is 110000
key string at 51 is 101000
key string at 52 is 101000
key string at 53 is 101000
key string at 54 is 110000
key string at 55 is 110000
key string at 56 is 110000
key string at 57 is 100000
key string at 58 is 110000
key string at 59 is 100000
key string at 60 is 100000
key string at 61 is 110000
key string at 62 is 100000
key string at 63 is 101000
key string at 64 is 110010
key string at 65 is 110000
key string at 66 is 101100
k

In [272]:
cap.release()